# Explore Hacker News Data - Delta Lake

**IMT Atlantique - Data Large Scale Project**

This notebook shows how to query Bronze and Silver Delta Lake tables.

## 1. Setup Spark with Delta Lake

In [ ]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

# Create Spark session with Delta Lake
builder = SparkSession.builder \
    .appName("Explore HN Data") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print("✓ Spark session created with Delta Lake support")

## 2. Load Bronze Data (Raw from Kafka)

In [3]:
# Load Bronze stories
bronze_stories = spark.read.format("delta").load("data/bronze/stories")
print(f"Bronze Stories: {bronze_stories.count()} records")
bronze_stories.show(5, truncate=False)

Bronze Stories: 29 records
+--------+----------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+-----------------------------------------------------------------------------+-----+---------------------------------------------------------------------+----+-------------+----------------+--------------------------+
|id      |by        |descendants|kids                                                                                                                                                                                                     

In [4]:
# Load Bronze comments
bronze_comments = spark.read.format("delta").load("data/bronze/comments")
print(f"Bronze Comments: {bronze_comments.count()} records")
bronze_comments.show(5, truncate=False)

Bronze Comments: 577 records
+--------+----------+--------------------+--------+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## 3. Load Silver Data (Cleaned)

In [5]:
# Load Silver stories
silver_stories = spark.read.format("delta").load("data/silver/stories")
print(f"Silver Stories: {silver_stories.count()} records")
silver_stories.show(5, truncate=False)

Silver Stories: 29 records
+--------+------------+-----------------------------------------------+-----------------------------------------------------------------+-----+-------------+--------+----------+-------------------+-------+--------+-----+--------------------------+
|id      |author      |title                                          |url                                                              |score|comment_count|text_raw|text_clean|timestamp          |has_url|has_text|type |_bronze_ingested_at       |
+--------+------------+-----------------------------------------------+-----------------------------------------------------------------+-----+-------------+--------+----------+-------------------+-------+--------+-----+--------------------------+
|46553556|speckx      |Test your square brackets                      |https://fluca1978.github.io/2025/12/10/testAndSquareBrackets.html|38   |24           |        |          |2026-01-09 14:23:44|true   |false   |story|2026-01-1

In [6]:
# Load Silver comments
silver_comments = spark.read.format("delta").load("data/silver/comments")
print(f"Silver Comments: {silver_comments.count()} records")
silver_comments.show(5, truncate=False)

Silver Comments: 576 records
+--------+-------------+--------+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+--------+----------+----------+-----------+----------+-------+-------------+-------+-------------------------+
|id      |author       |story_id|parent  |text_raw                                                                                                                                                                                                                     |text_clean                                                                                                                                               |time

## 4. View Schema

In [7]:
print("=== Silver Stories Schema ===")
silver_stories.printSchema()

=== Silver Stories Schema ===
root
 |-- id: integer (nullable = true)
 |-- author: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- text_raw: string (nullable = true)
 |-- text_clean: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- has_url: boolean (nullable = true)
 |-- has_text: boolean (nullable = true)
 |-- type: string (nullable = true)
 |-- _bronze_ingested_at: timestamp (nullable = true)



In [8]:
print("=== Silver Comments Schema ===")
silver_comments.printSchema()

=== Silver Comments Schema ===
root
 |-- id: integer (nullable = true)
 |-- author: string (nullable = true)
 |-- story_id: integer (nullable = true)
 |-- parent: integer (nullable = true)
 |-- text_raw: string (nullable = true)
 |-- text_clean: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- has_text: boolean (nullable = true)
 |-- word_count: integer (nullable = true)
 |-- char_count: integer (nullable = true)
 |-- has_replies: boolean (nullable = true)
 |-- is_deleted: boolean (nullable = true)
 |-- is_dead: boolean (nullable = true)
 |-- quality_score: double (nullable = true)
 |-- type: string (nullable = true)
 |-- _bronze_ingested_at: timestamp (nullable = true)



## 5. Statistics

In [9]:
# Story statistics
print("=== Story Statistics ===")
silver_stories.select("score", "comment_count").describe().show()

=== Story Statistics ===
+-------+------------------+------------------+
|summary|             score|     comment_count|
+-------+------------------+------------------+
|  count|                29|                29|
|   mean|159.27586206896552| 136.6206896551724|
| stddev|192.79372703038345|354.89992692036276|
|    min|                 5|                 0|
|    max|               748|              1936|
+-------+------------------+------------------+



In [10]:
# Comment statistics
print("=== Comment Statistics ===")
silver_comments.select("word_count", "quality_score").describe().show()

=== Comment Statistics ===
+-------+-----------------+-------------------+
|summary|       word_count|      quality_score|
+-------+-----------------+-------------------+
|  count|              576|                576|
|   mean|50.60069444444444| 0.9381944444444446|
| stddev|54.65070363876903|0.18593931792843288|
|    min|                1|                0.2|
|    max|              383|                1.0|
+-------+-----------------+-------------------+



## 6. Example Queries

In [11]:
# Top 10 stories by score
print("=== Top 10 Stories by Score ===")
silver_stories.orderBy("score", ascending=False) \
    .select("title", "score", "comment_count", "author") \
    .show(10, truncate=False)

=== Top 10 Stories by Score ===
+------------------------------------------------------------------------+-----+-------------+--------------+
|title                                                                   |score|comment_count|author        |
+------------------------------------------------------------------------+-----+-------------+--------------+
|Claude Cowork exfiltrates files                                         |748  |330          |takira        |
|Ask HN: Share your personal website                                     |703  |1936         |susam         |
|The URL shortener that makes your links look as suspicious as possible  |539  |108          |dreadsword    |
|A letter to those who fired tech writers because of AI                  |246  |162          |theletterf    |
|Ask HN: How are you doing RAG locally?                                  |242  |100          |tmaly         |
|Scaling long-running autonomous coding                                  |231  |142     

In [12]:
# Top 10 most active authors (by comment count)
print("=== Top 10 Most Active Authors ===")
from pyspark.sql.functions import count

silver_comments.groupBy("author") \
    .agg(count("*").alias("comment_count")) \
    .orderBy("comment_count", ascending=False) \
    .show(10)

=== Top 10 Most Active Authors ===
+--------------+-------------+
|        author|comment_count|
+--------------+-------------+
|          NULL|           12|
|         Havoc|            5|
|   Barathkanna|            4|
|    reactordev|            4|
|      torginus|            3|
|    prakashn27|            2|
|        Nora23|            2|
|       nottorp|            2|
|        simonw|            2|
|dfajgljsldkjag|            2|
+--------------+-------------+
only showing top 10 rows



In [13]:
# High quality comments (score > 0.9)
print("=== High Quality Comments ===")
silver_comments.filter("quality_score > 0.9") \
    .select("author", "word_count", "quality_score", "text_clean") \
    .show(5, truncate=80)

=== High Quality Comments ===
+-------------+----------+-------------+--------------------------------------------------------------------------------+
|       author|word_count|quality_score|                                                                      text_clean|
+-------------+----------+-------------+--------------------------------------------------------------------------------+
|         eajr|         9|          1.0|                             Local LibreChat which bundles a vector db for docs.|
|rahimnathwani|        13|          1.0|If your data aren't too large, you can use faiss-cpu and pickle\n\nhttps://py...|
|      motakuk|         8|          1.0|                                   LightRAG, Archestra as a UI with LightRAG mcp|
|       blahaj|        30|          1.0|I found out today that the location header of an HTTP redirect can be a tel:+...|
|  simonsarris|        51|          1.0|https://simonsarris.com - My site\n\nhttps://map.simonsarris.com - My newsle

In [14]:
# Join stories with their comments
print("=== Stories with Comment Stats ===")
from pyspark.sql.functions import avg, count

comment_stats = silver_comments.groupBy("story_id") \
    .agg(
        count("*").alias("actual_comments"),
        avg("word_count").alias("avg_word_count"),
        avg("quality_score").alias("avg_quality")
    )

stories_with_stats = silver_stories.join(
    comment_stats,
    silver_stories.id == comment_stats.story_id,
    "left"
).select(
    silver_stories.title,
    silver_stories.score,
    comment_stats.actual_comments,
    comment_stats.avg_word_count,
    comment_stats.avg_quality
)

stories_with_stats.show(10, truncate=False)

=== Stories with Comment Stats ===
+-------------------------------------------------------------------------------+-----+---------------+------------------+------------------+
|title                                                                          |score|actual_comments|avg_word_count    |avg_quality       |
+-------------------------------------------------------------------------------+-----+---------------+------------------+------------------+
|Test your square brackets                                                      |38   |8              |49.125            |1.0               |
|San Remo Pasta Measurer                                                        |28   |8              |28.0              |1.0               |
|Bare metal programming with RISC-V guide (2023)                                |47   |3              |60.333333333333336|0.8333333333333334|
|Z80 Mem­ber­ship Card                                                          |52   |9              |50.2222222

## 7. Delta Lake Time Travel

In [15]:
# View transaction history
print("=== Delta Lake Transaction History ===")
dt = DeltaTable.forPath(spark, "data/bronze/stories")
dt.history().select("version", "timestamp", "operation", "operationMetrics").show(truncate=False)

=== Delta Lake Transaction History ===
+-------+-----------------------+---------+-------------------------------------------------------------+
|version|timestamp              |operation|operationMetrics                                             |
+-------+-----------------------+---------+-------------------------------------------------------------+
|0      |2026-01-15 16:11:11.379|WRITE    |{numFiles -> 1, numOutputRows -> 29, numOutputBytes -> 21666}|
+-------+-----------------------+---------+-------------------------------------------------------------+



In [ ]:
# Query previous version (if exists)
print("=== Query Version 0 ===")
df_v0 = spark.read.format("delta").option("versionAsOf", 0).load("data/bronze/stories")
print(f"Version 0 had {df_v0.count()} records")

## 8. SQL Queries (Alternative)

In [ ]:
# Register as SQL tables
silver_stories.createOrReplaceTempView("stories")
silver_comments.createOrReplaceTempView("comments")

print("✓ Registered SQL tables: stories, comments")

In [ ]:
# SQL query example
spark.sql("""
    SELECT 
        s.title,
        s.score,
        COUNT(c.id) as comment_count,
        AVG(c.quality_score) as avg_quality
    FROM stories s
    LEFT JOIN comments c ON s.id = c.story_id
    GROUP BY s.title, s.score
    ORDER BY s.score DESC
    LIMIT 10
""").show(truncate=False)

## 9. Convert to Pandas (for visualization)

In [ ]:
# Convert to Pandas for plotting
stories_pd = silver_stories.toPandas()
comments_pd = silver_comments.toPandas()

print(f"Stories: {len(stories_pd)} rows")
print(f"Comments: {len(comments_pd)} rows")

# Preview
stories_pd.head()

In [16]:
# Example visualization with seaborn
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.histplot(comments_pd['quality_score'], bins=20, kde=True)
plt.title('Distribution of Comment Quality Scores')
plt.xlabel('Quality Score')
plt.ylabel('Frequency')
plt.show()

NameError: name 'comments_pd' is not defined

<Figure size 1000x600 with 0 Axes>

## 10. Cleanup

In [ ]:
# Stop Spark session when done
spark.stop()
print("✓ Spark session stopped")